In [5]:
import os
import re

import unicodedata
import time
import pandas as pd
import numpy as np
from rapidfuzz import process, fuzz
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
from transformers import logging as transformers_logging
from tabulate import tabulate
transformers_logging.set_verbosity_error()
import json

C:\Users\sandr\UntrackedFolder\movie-bot\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
stop_words_to_keep = ["what", "when", "where", "which", "while", "who", "whom", "why", "with", "how", "before", "after","same"]
stop_words = set([s for s in stopwords.words('english') if s not in stop_words_to_keep])

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sandr\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
class NERParser:
    def __init__(self, model_name: str = "dslim/bert-base-NER", lowercase: bool = False):
        """
        Initialize the NER parser with a model and optionally configure the lowercase preprocessing.
        """
        self.model_name = model_name
        self.lowercase = lowercase
        self.device = self.get_device()
        
        # Load the tokenizer and model
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_name, do_lower_case=self.lowercase)
        self.model = AutoModelForTokenClassification.from_pretrained(self.model_name)
        
        # Set up the NER pipeline
        self.nlp_pipeline = pipeline(
            "ner", 
            model=self.model, 
            tokenizer=self.tokenizer, 
            device=self.device, 
            aggregation_strategy="simple"
        )

    def get_device(self):
        """
        Determines whether to use MPS, CUDA, or CPU depending on the available hardware.
        """
        if torch.backends.mps.is_available():
            print("MPS device found, using MPS backend.\n")
            return torch.device("mps")
        elif torch.cuda.is_available():
            print(f"CUDA device found, using CUDA backend. Device: {torch.cuda.get_device_name(0)}\n")
            return torch.device("cuda")
        else:
            print("Neither MPS nor CUDA found, using CPU.\n")
            return torch.device("cpu")

    
    def parse_ner_results(self, ner_results: list):
        """
        Parse the NER results and extract entities related to 'PER' (persons) and 'MISC' (potential movie titles).
        """
        per_entities, misc_entities = [], []
        
        for entity in ner_results:
            # Extraction of all Persons
            if entity['entity_group'] == 'PER':
                per_entities.append(entity['word'])
            # Extraction of all Misc that could indicate movies
            elif entity['entity_group'] == 'MISC':
                misc_entities.append(entity['word'])
        
        return per_entities, misc_entities

    
    def process_query(self, query: str):
        """
        Processes a text query, runs NER, and returns the extracted actors and movie names.
        """
        # Optionally lowercase the input if configured
        if self.lowercase:
            query = query.lower()
        
        # Run the NER pipeline
        ner_results = self.nlp_pipeline(query)

        # Parse the results to extract actors and movies
        per_entities, misc_entities = self.parse_ner_results(ner_results)
        
        return per_entities, misc_entities

ner_parser = NERParser(lowercase=False)

Neither MPS nor CUDA found, using CPU.



In [8]:
class DataBase:
    """Handles the extraction of context data for given people and movies from a database, with fuzzy matching for names."""

    def __init__(self):
        self.db = pd.read_pickle(os.path.join(os.getcwd(), r"exports/extended_graph_triples.pkl"))
        
        with open(r"exports/entity_db.json", encoding="utf-8") as f: 
            self.entities = json.load(f)
            self.entity_list = list(subject.lower() for subject, types in self.entities.values())
        
        # otherwise exact matching will fail
        self.db['subject_id'] = self.db['subject_id'].astype(str).str.strip()

    @staticmethod
    def normalize_string(s):
        """Cleans the input entity to a uniform naming convention, by removing non ascii characters, encoding it to utf, setting it to lowercase, and removing redundant spaces"""
        s = s.lower()
        s = unicodedata.normalize('NFKD', s)
        s = s.encode('ascii', 'ignore').decode('utf-8')
        s = re.sub(r'[^\w\s]', '', s)
        s = ' '.join(s.split())
        return s
    
    def fetch(self, entity_lst, search_column):
        
        relevant = self.db[self.db[search_column].isin(entity_lst)].dropna(axis=1)
           
        if relevant.empty:
            print(f"No context data found for given information.")
            return pd.DataFrame()          
        
        pivot_df = relevant.pivot_table(
            index='subject_id',
            columns='predicate_label',
            values='object_label',
            aggfunc=lambda x: ' | '.join(x.astype(str))
        )
    
        pivot_df.reset_index(inplace=True)
    
        return pivot_df

In [9]:
class QueryEmbedder:
    
    def __init__(self):
        self._glove_embeddings = self._load_glove_embeddings("exports/glove.6B/glove.6B.300d.txt")
        
    def _load_glove_embeddings(self, file_path):
        embeddings = {}
        with open(file_path, 'r', encoding='utf-8') as f:
            for line in f:
                values = line.split()
                word = values[0]
                vector = np.asarray(values[1:], dtype='float32')
                embeddings[word] = vector
        return embeddings
    
    def embed_phrase(self, phrase):
        words = phrase.split()
        word_vectors = [self._glove_embeddings[word.lower()] for word in words if word.lower() in self._glove_embeddings]
        
        if len(word_vectors) == 0:
            return np.zeros(300)
        
        return np.mean(word_vectors, axis=0)
    

In [10]:
class LLM():
    
    def __init__(self):
        self.qa_model = pipeline("question-answering", model="distilbert-base-cased-distilled-squad", top_k=1)
    
    def _query(self, query, context_df):
        
        context = ""
        for index, row in context_df.iterrows():
            row_context = " ".join([f"{col}: {row[col]}" for col in context_df[top_columns].columns])
            context += row_context + " "
        
        output = self.qa_model(question=query, context=context)
        
        answer_str = str()
        if isinstance(output, list):
            answer_str = ", ".join([result['answer'] for result in output])
            
        elif isinstance(output, dict):
            answer_str = output['answer']
        
        if not answer_str:
            answer_str = "No answer found."
        
        return answer_str

In [12]:
def cosine_sim(vec1, vec2):
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    
    if norm_vec1 == 0 or norm_vec2 == 0:
        return 0.0 
    
    return np.dot(vec1, vec2) / (norm_vec1 * norm_vec2)

def find_closest_columns(query_embeddings, column_embeddings, top_n=5):
    column_similarities = {}

    for col, col_vec in column_embeddings.items():
        similarities = [cosine_sim(col_vec, q_vec) for q_vec in query_embeddings if np.linalg.norm(q_vec) > 0]
        column_similarities[col] = np.mean(similarities) if similarities else -1

    sorted_columns = sorted(column_similarities.items(), key=lambda item: item[1], reverse=True)
    
    return [col for col, sim in sorted_columns[:top_n]]

In [13]:
db = DataBase()

qe = QueryEmbedder()

llm = LLM()

In [20]:
def fuzzy_match(query_str, comparison_list, threshold=30):
    matches = process.extract(query_str, comparison_list, scorer=fuzz.partial_ratio, limit=100)
    
    id_name_score = []
    for match in matches:
        name = match[0]
        score = match[1]
        matched_id = next(key for key, value in db.entities.items() if value[0] == name)
            
        length_ratio = len(name) / len(query_str)
        adjusted_score = score * length_ratio
                
        id_name_score.append((matched_id, name, adjusted_score))
        
    print(tabulate(id_name_score[:3], headers=["id", "Name", "Score"], tablefmt="grid"))
    
    return [id for id, _, score in id_name_score if score >= threshold]

In [21]:
# query = "Who directed The Bridge on the River Kwai?"
query = "Who is the director of Good Will Hunting?"

normalized_query = query.lower()

index_matches = fuzzy_match(query, db.entity_list, threshold=30)

context = db.fetch(index_matches, "subject_id")

if context.empty:
    print("No context data found for given IDs or string")

    context = pd.DataFrame()

context.head()

+-----------------------------------------+--------+---------+
| id                                      | Name   |   Score |
+=========================================+========+=========+
| http://www.wikidata.org/entity/Q214227  | ti     | 4.87805 |
+-----------------------------------------+--------+---------+
| http://www.wikidata.org/entity/Q1501773 | o      | 2.43902 |
+-----------------------------------------+--------+---------+
| http://www.wikidata.org/entity/Q158697  | ing    | 7.31707 |
+-----------------------------------------+--------+---------+


predicate_label,subject_id,aspect ratio,assessment,award received,box office,cast member,color,country of origin,ddis:rating,ddis:tag,...,narrative location,nmhh film rating,node description,node label,nominated for,notable work,original language of film or tv show,production company,publication date,screenwriter
0,http://www.wikidata.org/entity/Q193835,widescreen,"reverse bechdel test, bechdel test","academy award for best supporting actor, acade...",225900000,"harmony korine, stellan skarsgard, ben affleck...",color,united states of america,8.3,"inspiring, feel-good, humor, flashback, psycho...",...,boston,category iv,1997 film by Gus Van Sant,Good Will Hunting,"academy award for best film editing, academy a...",minnie driver,english,"miramax, a band apart",1997-12-05,"matt damon, ben affleck"


In [22]:
column_embeddings = {col: qe.embed_phrase(col) for col in context.columns}

query_embeddings = [qe.embed_phrase(word) for word in query]


In [23]:
top_columns = find_closest_columns(query_embeddings, column_embeddings, top_n=10)
filtered_context_df = context[top_columns]

In [24]:
llm._query(query, filtered_context_df)

'Gus Van Sant'